# Introduction
This notebook performs some basic Machine Learning analysis of the New York Rolling Sales data from the [Doing Data Science : Straight Talk from the Frontline book][booklink] by Cathy O'Neil & Rachel Schutt published by O'Reilly Media. The data can be downloaded [here][datalink] and specifically I will be briefly looking at the `rollingsales_manhattan.xls` data.

The file contains housing sales data for Manhattan properties, including the neighbourhood, building class (family home, condo, etc), sale price, sale date, etc. 

I will be doing the data analysis using Julia.

[booklink]: https://www.oreilly.com/library/view/doing-data-science/9781449363871/
[datalink]: https://github.com/oreillymedia/doing_data_science

## Data Cleaning
I'll be performing some data cleaning steps that I previously performed in another notebook. These have been detailed [here:](https://mybinder.org/v2/gh/ncalvertuk/DoingDataScienceNbs_Julia/master?filepath=Chapter2_RollingSales.ipynb)

In [ ]:
using DataFrames, Gadfly, Queryverse, Statistics, Dates,Plots,StatsPlots,VegaDatasets,CSV, JSON
plotly()
Plots.PlotlyBackend()
D = DataFrame(load("./rollingsales_manhattan.xls", "Manhattan!A5:U27400"));
# Clean up the property names
for pn in propertynames(D)
  rename!(D,pn => Symbol(filter(x -> !isspace(x),string(pn))))
  rename!(D,pn => Symbol(replace(string(pn),"-"=> "")))
end
# Make the strings lower case.
D = D |> @mutate(NEIGHBORHOOD = lowercase(join(split(strip(_.NEIGHBORHOOD)))),BUILDINGCLASSCATEGORY = lowercase(join(split(strip(_.BUILDINGCLASSCATEGORY)))),ADDRESS = lowercase(_.ADDRESS),EASEMENT = lowercase(join(split(strip(_.EASEMENT)))),BUILDINGCLASSATPRESENT = lowercase(join(split(strip(_.BUILDINGCLASSATPRESENT)))),BUILDINGCLASSATTIMEOFSALE = lowercase(join(split(strip(_.BUILDINGCLASSATTIMEOFSALE))))) |> DataFrame
# Keep family homes with a sale price of more than $100k and with a gross square foot >0
D_fam = D |> @filter(_.SALEPRICE > 100000 && _.GROSSSQUAREFEET >0) |> @mutate(PRICEPERSQF = _.SALEPRICE/_.GROSSSQUAREFEET) |> @mutate(FAMILYHOME = occursin("family",_.BUILDINGCLASSCATEGORY)) |> @filter(_.FAMILYHOME == true)|> DataFrame

Now that we have the data, lets perform some linear regression. We'll once again start with a basic (and inaccurate!) model before increasing the number of features. Let's start with a single factor - the gross square foot.

First let's have a look at the data by histogramming the two variables and generating a scatter plot. We'll use a logarithmic scale

In [ ]:
#p1 = D_fam |> @vlplot(:bar, x={:GROSSSQUAREFEET, bin=true,scale={type="log",base=10}}, y="count()")
p1 = D_fam |> @vlplot(
    :bar,
    transform=[
        {calculate="log(datum.GROSSSQUAREFEET)/log(10)", as="log_x"},
        {field="log_x",bin={"maxbins"=30},as="bin_log_x"},
        {calculate="pow(10, datum.bin_log_x)", as="x1"},
        {calculate="pow(10, datum.bin_log_x_end)", as="x2"}
    ],
    x={"x1:q", scale={type="log",base=10},axis={tickCount=5},title="Gross Square Feet"},
    x2=:x2,
    y={aggregate="count",type="quantitative"}
)
p2 = D_fam |> @vlplot(
    :bar,
    transform=[
        {calculate="log(datum.SALEPRICE)/log(10)", as="log_x"},
        {field="log_x",bin={"maxbins"=30},as="bin_log_x"},
        {calculate="pow(10, datum.bin_log_x)", as="x1"},
        {calculate="pow(10, datum.bin_log_x_end)", as="x2"}
    ],
    x={"x1:q", scale={type="log",base=10},axis={tickCount=5},title="Sale Price (USD)"},
    x2=:x2,
    y={aggregate="count",type="quantitative"}
)
p3 = D_fam |> @vlplot(:circle,x={:GROSSSQUAREFEET,scale={type="log",base=10}},y={:SALEPRICE,scale={type="log",base=10}})
display(p1)
display(p2)
display(p3)

It is clear from the plots that there are a number of factors to consider here when trying to predict the sale price and just using the gross square footage will not lead to a satisfactory fit. In particular there distributions appear to be bimodal, particularly the sale price. Lets go ahead anyway!

# Linear Regression using MLJ
I'll be using [MLJ](https://alan-turing-institute.github.io/MLJ.jl/stable/) to perform the basic Machine Learning here. I first create a new DataFrame containing the variables I'm interested in and then use the unpack method to separate this out into X & y arrays. MLJ requires the input variables to be in a table format, I found I had to reshape X when it contained only a single variable.

After getting the data in the correct format I load in the model I'm interested in - in this case I'm using Linear Regression. I then instantiate the model, setting the intercept to 0 before training the model and predicting on my test set.

In [ ]:
using MLJ,MLJLinearModels
D_fam_sub = D_fam |> @map({X = log(_.GROSSSQUAREFEET),Y = log(_.SALEPRICE)}) |> DataFrame
y,X = unpack(D_fam_sub,==(:Y),==(:X);:X=>Continuous,:Y=>Continuous)
X = MLJ.table(reshape(X,291,1))
@load LinearRegressor pkg="MLJLinearModels"
model = LinearRegressor(fit_intercept = false)
LR = machine(model, X, y)
train, test = partition(eachindex(y), 0.7, shuffle=true, rng=1234); # 70:30 split
fit!(LR, rows=train)
yhat = predict(LR, rows=test);


To evaluate the model I can use the evaluate function, notice that I call evaluate! as I'm applying it to an existing model. I also define rsq as a new evaluation metric, which is then used in the evaluate! function. The evaluate function uses cross-validation as default, with 6 folds.

In [ ]:
rsq(y_hat,y) = 1 - sum((y .- y_hat).^2)/sum((y.-mean(y)).^2)
evaluate!(LR,measure=[l2,rms,rmslp1,rsq])

As expected, the model is pretty useless! Let's get the fitted parameters, generate a line from these and then add this to a plot along with the original data points

In [ ]:
x1 = range(0,stop=9.5,length=1000)
y1 = x1.*fitted_params(LR).coefs[1][2]

p = Plots.plot(exp.(reshape(D_fam_sub[!,1],291,1)),exp.(reshape(D_fam_sub[!,2],291,1)), seriestype = :scatter,size = (400,400),xlabel="GROSSSQUAREFEET (sqf)", ylabel = "SALEPRICE (USD)",title="Single Variable Fit")
Plots.plot!(p,exp.(x1),exp.(y1))


We can also histogram the residuals, unlike in the last plot we won't transform them by taking the exponential.

In [ ]:
using KernelDensity,NumericalIntegration
U = kde(y[test]-yhat)
p1 = histogram(y[test]-yhat,bins= 30,title ="Residual Histogram",size = (300,200))
mult = (0.2*87)./integrate(U.x,U.density) # histo bins are 0.2 wide, so we need to scale the density to match the plot
Plots.plot!(p1,U.x,mult .*U.density)
display(p1)

Pretty naff, let's see if we can improve this by increasing the number of input variables.

In [ ]:
D_fam_sub = D_fam |> @map({X = log(_.GROSSSQUAREFEET),X1 = log(_.LANDSQUAREFEET),Y = log(_.SALEPRICE)}) |> DataFrame
y,X = unpack(D_fam_sub,==(:Y),!=(:Y));
LR1 = machine(model, X, y)
train, test = partition(eachindex(y), 0.7, shuffle=true, rng=1234); # 70:30 split
fit!(LR1, rows=train)
yhat1 = predict(LR1, rows=test);
evaluate!(LR1,measure=[l2,rms,rmslp1,rsq])

In [ ]:
U1 = kde(y[test]-yhat1)
p2 = histogram(y[test]-yhat1,bins= 30,title ="Residual Histogram",size = (300,200))
mult = (0.2*87)./integrate(U1.x,U1.density) # histo bins are 0.2 wide, so we need to scale the density to match the plot
Plots.plot!(p2,U.x,mult .*U1.density)
display(p2)

Still a bit naff, not to be unexpected given that the price varies between neighborhoods as we've seen previously. Let's include this as a categorical variable.

In [ ]:
D_fam_sub = D_fam |> @map({X = log(_.GROSSSQUAREFEET),X1 = log(_.LANDSQUAREFEET),X2 = _.NEIGHBORHOOD,Y = log(_.SALEPRICE)}) |> DataFrame
y,X = unpack(D_fam_sub,==(:Y),!=(:Y))
schema(X)

We can see that the Neighbourhood has a ```Textual``` type, we want this to be a ```Multiclass``` type (with no ordering). We use ```coerce``` to force this

In [ ]:
coerce!(X,:X2=>Multiclass)
schema(X)

Now we implement the One Hot Encoder to convert from categorical variables to continuous. We create a new variable for each neighborhood, then each data row contains a 1 in the variable relating to the neighborhood and 0 in the others.

In [ ]:
hot = OneHotEncoder(ordered_factor=false);
mach = fit!(machine(hot, X))
X = transform(mach, X)
schema(X)

Lets take a look at the first data row. As we can see, X2__alphabetcity = 1 and the other values are all 0 (Except Gross Square Feet and Land Square Feet).

In [ ]:
first(X,1) |> pretty

In [ ]:
LR2 = machine(model, X, y)
train, test = partition(eachindex(y), 0.7, shuffle=true, rng=1234); # 70:30 split
fit!(LR2, rows=train)
yhat2 = predict(LR2, rows=test);
evaluate!(LR2,measure=[l2,rms,rmslp1,rsq])

In [ ]:
U2 = kde(y[test]-yhat2)
p3 = histogram(y[test]-yhat2,bins= range(-2,stop=10,length=51),title ="Residual Histogram",size = (300,200))
mult = (0.24*87)./integrate(U2.x,U2.density) # histo bins are 0.2 wide, so we need to scale the density to match the plot
Plots.plot!(p3,U2.x,mult .*U2.density)
display(p3)

In [ ]:
all_resids = [y[test]-yhat, y[test]-yhat1,y[test]-yhat2]
p4 = histogram(all_resids,bins= range(-2,stop=10,length=51),title ="Residual Histogram",size = (300,200))

From the plot above it appears that the residuals are generally decreased when using the neighborhood however there are some outliers with large residuals. We can also see from the Cross-validation that the MSE varies wildly from 0.4 up to 60(!). This is most likely caused by the low numbers of properties in certain neighborhoods. We can see this in the table below.

A solution to this could be to combine different neighbourhoods into larger geographical regions although this would have to be done carefully in case there were large differences in the average price per sqf in two neighbouring neighbourhoods, for example.

In [ ]:
println(D_fam |>
    @groupby(_.NEIGHBORHOOD) |>
    @map({Key=key(_), Count=length(_)}) |>
    DataFrame)

Finally, we'll add in one extra variable - Building class. This is another categorical variable, so we'll use One Hot Encoding once again.

In [ ]:
D_fam_sub = D_fam |> @map({X = log(_.GROSSSQUAREFEET),X1 = log(_.LANDSQUAREFEET),X2 = _.NEIGHBORHOOD,X3 = _.BUILDINGCLASSCATEGORY,Y = log(_.SALEPRICE)}) |> DataFrame
y,X = unpack(D_fam_sub,==(:Y),!=(:Y))
coerce!(X,:X2=>Multiclass)
coerce!(X,:X3=>Multiclass)
mach = fit!(machine(hot, X))
X = transform(mach, X)
schema(X)

In [ ]:
LR3 = machine(model, X, y)
train, test = partition(eachindex(y), 0.7, shuffle=true, rng=1234); # 70:30 split
fit!(LR3, rows=train)
yhat3 = predict(LR3, rows=test);
evaluate!(LR3,measure=[l2,rms,rmslp1,rsq])

In [ ]:
U3 = kde(y[test]-yhat3)
p4 = histogram(y[test]-yhat3,bins= range(-2,stop=10,length=51),title ="Residual Histogram",size = (300,200))
mult = (0.24*87)./integrate(U3.x,U3.density) # histo bins are 0.2 wide, so we need to scale the density to match the plot
Plots.plot!(p4,U3.x,mult .*U3.density)
display(p4)

In [ ]:
all_resids = [y[test]-yhat, y[test]-yhat1,y[test]-yhat2,y[test]-yhat3]
p4 = histogram(all_resids,bins= range(-2,stop=10,length=51),title ="Residual Histogram",size = (300,200))

This appears to have improved things somewhat compared to the previous fit. We can also interact the last two variables. Let's try that before moving on.

In [ ]:
D_fam_sub = D_fam |> @map({X = log(_.GROSSSQUAREFEET),X1 = log(_.LANDSQUAREFEET),X2 = string(_.NEIGHBORHOOD,_.BUILDINGCLASSCATEGORY),Y = log(_.SALEPRICE)}) |> DataFrame
y,X = unpack(D_fam_sub,==(:Y),!=(:Y))
coerce!(X,:X2=>Multiclass)
mach = fit!(machine(hot, X))
X = transform(mach, X)
schema(X)

In [ ]:
LR4 = machine(model, X, y)
train, test = partition(eachindex(y), 0.7, shuffle=true, rng=1234); # 70:30 split
fit!(LR4, rows=train)
yhat4 = predict(LR4, rows=test);
evaluate!(LR4,measure=[l2,rms,rmslp1,rsq])

In [ ]:
U4 = kde(y[test]-yhat4)
p5 = histogram(y[test]-yhat4,bins= range(-2,stop=10,length=51),title ="Residual Histogram",size = (300,200))
mult = (0.24*87)./integrate(U4.x,U4.density) # histo bins are 0.2 wide, so we need to scale the density to match the plot
Plots.plot!(p5,U4.x,mult .*U4.density)
display(p5)

In [ ]:
all_resids = [y[test]-yhat, y[test]-yhat1,y[test]-yhat2,y[test]-yhat3,y[test]-yhat4]
p6 = histogram(all_resids,bins= range(-2,stop=10,length=51),title ="Residual Histogram",size = (300,200))

This leads to more extreme residual values than the previous model. Again, by aggregating the neighbourhoods we may be able to overcome this.

## Classifying Neighbourhood using Latitude & Longitude
The second part of the exercise was to predict the neighbourhood of properties, implementing a k-nearest neighbours classifier based on the latitude and longitude. This should be relatively straightforward but will require some data cleaning before getting started.

Firstly we'll only keep properties that were sold with a price of > $10,000

In [ ]:
D_sub = D |> @filter(_.SALEPRICE > 10000 && _.GROSSSQUAREFEET >0)|> DataFrame

Lets take a look at the addresses to see if there's any problems. As we can see there is a rather large space at the end of them, so we need to get rid of that. There's also some extra spaces before the word 'street' in some cases. Lets also add in the city and state (both New York in this case), the zip code, and the country (US).

In [ ]:
D_sub[!,:ADDRESS]

In [ ]:
D_sub = D_sub |> @mutate(LONGADDRESS  = replace.(_.ADDRESS,"  " => "")) |> @mutate(LONGADDRESS  = string(_.LONGADDRESS,", New York, NY, ", Int(_.ZIPCODE), " US")) |> @mutate(LONGADDRESS  = replace.(_.LONGADDRESS," , " => ", ")) |> DataFrame

We'll create a new DataFrame solely containing the unique addresses and neighborhoods. We'll extend this with the lats & longs when we've got them.

In [ ]:
D_adds = D_sub |> @groupby(_.LONGADDRESS) |>
    @map({LONGADDRESS=key(_), NEIGHBORHOOD=first(unique(_.NEIGHBORHOOD))}) |>
    DataFrame

To get the lats & longs I'll use [LocationIQ](https://locationiq.com/). You'll need to grab a free API key to use this, which limits the number of requests to 60 per minute. For our 1450 unique addresses this is going to take 24-25 minutes in total. We'll need to use the [HTTP.jl](https://github.com/JuliaWeb/HTTP.jl) package to send the requests.

In [ ]:
using ProgressMeter
using HTTP
using JSON
lats =[]
lons = []
apikey = "YOUR_API_KEY"
@showprogress for add in D_adds[!,:LONGADDRESS]
  Add = replace.(replace.(add,","=>"")," "=>"%20")
  url1 = string("https://us1.locationiq.com/v1/search.php?key=",apikey,"&q=",Add,"&format=json");
  try 
    r = HTTP.request("GET", url1)
  bod = JSON.parse(String(r.body))
  append!(lats,parse(Float64,bod[1]["lat"]))
   append!(lons,parse(Float64,bod[1]["lon"]))
  sleep(1)
  catch
    append!(lats,NaN)
   append!(lons,NaN)
end

We want to append the Latitude & Longitude values on to our DataFrame and then remove any NaN values (there was only 1 when I searched).

In [ ]:
D_adds[!,:LATITUDE] = lats;
D_adds[!,:LONGITUDE] = lons;
D_adds = D_adds[findall(.!(isnan.(D_adds[!,:LATITUDE]))),:]

In [ ]:
using DataFrames, Gadfly, Queryverse, Statistics, Dates,Plots,StatsPlots, VegaDatasets,CSV, JSON
plotly()
Plots.PlotlyBackend()

Let's take a look at the values on a map, to ensure that the returned values were correct. To do this we'll import the ```us-10m``` dataset from ```VegaDatasets``` which contains the boundaries of the US states at the 1:10,000,000 scale. The excellent [Vega Projection Editor](https://vega.github.io/vega/docs/projections/) allowed me to customise the projection settings to ensure that the map was centred on Manhattan, but not too zoomed in to miss out any outlying data points.

In [ ]:
us10m = dataset("us-10m")
@vlplot(width=800, height=500) +
@vlplot(
    mark={
        :geoshape,
        fill="#eee",
        stroke=:white
    },
    data={
        values=us10m,
        format={
            type=:topojson,
            feature=:states
        }
    },
    projection={type=:azimuthalEqualArea,scale=2500,center=[29,43],rotate=[105,0,0]},
) +
@vlplot(
    :circle,
    data=D_adds,
    projection={type=:azimuthalEqualArea,scale=2500,center=[29,43],rotate=[105,0,0]},
    longitude={"LONGITUDE:q"},
   latitude={"LATITUDE:q"},
   size={value=5},
   color={value=:blue}
)

We can see a few incorrect data points, some are in upstate New York and some others are in Brooklyn. Hovering over some of the points with the mouse brings up the Lat/Lon values. We can see that those points with Longitude > -73.7 are on Brooklyn and Latitude > 41 are in upstate New York. Let's zoom in further and take a deeper look at Manhattan

In [ ]:
us10m = dataset("us-10m")
@vlplot(width=800, height=500) +
@vlplot(
    mark={
        :geoshape,
        fill="#eee",
        stroke=:white
   },
    data={
        values=us10m,
        format={
            type=:topojson,
            feature=:states
        }
    },
    projection={type=:azimuthalEqualArea,scale=15000,center=[29,40.5],rotate=[102.5,0,0]},
) +
@vlplot(
    :circle,
    data=D_adds,
    projection={type=:azimuthalEqualArea,scale=15000,center=[29,40.5],rotate=[102.5,0,0]},
    longitude={"LONGITUDE:q"},
    latitude={"LATITUDE:q"},
    size={value=5},
    color={value=:blue}
)

Zooming in allows us to identify extra mislocated properties. To separate out the incorrect locations I downloaded the boundary of New York City Boroughs from [NYC OpenData](https://data.cityofnewyork.us/City-Government/Borough-Boundaries/tqmj-j8zm). It is important to note here that the boundary is defined by 34 separate polygons as Manhattan includes a number of islands (e.g. the Statue of Liberty is on Liberty Island which is included as a separate polygon).

In [ ]:
using JSON
borofile = open("BoroughBoundaries.geojson")
headers = JSON.parse(read(borofile,String))
println(headers["features"][3]["properties"])
manh_boundary = headers["features"][3]["geometry"]["coordinates"]

I then created a new column in our DataFrame that will be true if the location is within Manhattan and false if it is not.

In [ ]:
D_adds[!,:MANHATTAN] = falses(1449);
nrs = size(D_adds)[1];

Using the [Luxor](https://github.com/JuliaGraphics/Luxor.jl) Package we can iterate through all of the boundary polygons and check whether each location is within the boundary of Manhattan. We'll then fill the relevant values in the D_adds DataFrame.

In [ ]:
using Luxor
using ProgressMeter
@showprogress for region in manh_boundary
  bnd = [Point(p[1],p[2]) for p in region[1]];
  for i in range(1,stop=nrs)
    coord = Point(D_adds[i,:LONGITUDE],D_adds[i,:LATITUDE]);
    isin = isinside(coord,bnd);
    if (isin==true)
      D_adds[i,:MANHATTAN] = true
    end
  end
end

Lets filter out the incorrect locations and plot them.

In [ ]:
D_adds = D_adds |> @filter(_.MANHATTAN >0) |> DataFrame;
@vlplot(width=800, height=500) +
@vlplot(
    mark={
        :geoshape,
        fill="#eee",
        stroke=:white
    },
    data={
        values=us10m,
        format={
            type=:topojson,
            feature=:states
        }
    },
    projection={type=:azimuthalEqualArea,scale=15000,center=[29,40.5],rotate=[102.5,0,0]},
) +
@vlplot(
    :circle,
    data=D_adds,
    projection={type=:azimuthalEqualArea,scale=15000,center=[29,40.5],rotate=[102.5,0,0]},
    longitude={"LONGITUDE:q"},
    latitude={"LATITUDE:q"},
    size={value=5},
    color={value=:blue}
)

In [ ]:
println(size(D_adds)[1])

We still have 1339 properties to use for our classification, which is a good amount. Let's prepare the data ready for passing into MLJ.

In [ ]:
using MLJ,MLJLinearModels
D_adds_sub = D_adds |> @map({X = _.LONGITUDE,X1 = _.LATITUDE,Y = _.NEIGHBORHOOD}) |> DataFrame 
y,X = unpack(D_adds_sub,==(:Y),!=(:Y));
y=categorical(y)

We can search for the KNN model using the following command, handy if we don't know the exact name of the model.

In [ ]:
models("Neighbors")

Let's set up a KNN and increase the number of neighbours to use for the classification. We should use Cross Validation here, however I'm omitting for this simple example. I also had a few issues in testing where the CV was not selecting at least 1 of each type in the training sets the evaluate function was giving out an error.

In [ ]:
function knn_acc(yhat,y)
  return sum(yhat .== y)/length(y)
end
nbrs = 1:10
accs = []
@load KNeighborsClassifier pkg="ScikitLearn"
for k in nbrs
model = KNeighborsClassifier(n_neighbors = k);
knn = machine(model, X, y);
train, test = partition(eachindex(y), 0.7, shuffle=true, rng=1234); # 70:30 split
fit!(knn, rows=train);
yhat = predict_mode(knn, rows=test);
append!(accs,knn_acc(yhat,y[test]));
end

Finally, let's plot the accuracy at each value of k. As we can see a k = 1 provides the highest accuracy in this case, which may be somewhat surprising, however this is an overly simplified example.

In [ ]:
Plots.plot(nbrs,accs,seriestype=:scatter,xlabel="n_neighbours", ylabel = "Accuracy",size = (300,300),ylim = [0,1])